In [ ]:
# Importa el módulo necesario para interactuar con Google Drive desde Google Colab
from google.colab import drive

# Monta Google Drive, permitiendo el acceso a los archivos almacenados en Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ruta del archivo en Google Drive
file_path = '/content/drive/MyDrive/PIA Optimizacion.xlsx'

In [ ]:
# Importe de librerias
import pandas as pd

In [ ]:
# Función para cargar los datos desde un archivo Excel
def cargar_datos(file_path):
    # Carga el archivo Excel completo
    excel_data = pd.ExcelFile(file_path)

    # Lee la hoja 'Distancias' del archivo Excel y la carga en un DataFrame de pandas
    distances_df = pd.read_excel(file_path, sheet_name='Distancias')

    # Lee la hoja 'Demanda' del archivo Excel y la carga en un DataFrame de pandas
    demanda_df = pd.read_excel(file_path, sheet_name='Demanda')

    # Retorna dos DataFrames: uno con los datos de distancias y otro con los datos de demanda
    return distances_df, demanda_df

In [ ]:
# Función para limpiar y convertir los datos de demanda
def limpiar_convertir_demanda(demanda_df):
    # Elimina la cadena ' kg' de la columna 'Cantidad a Surtir' y convierte los valores a enteros
    demanda_df['Cantidad a Surtir'] = demanda_df['Cantidad a Surtir'].str.replace(' kg', '').astype(int)

    # Establece la columna 'Sucursales' como índice y convierte la columna 'Cantidad a Surtir' en un diccionario
    return demanda_df.set_index('Sucursales')['Cantidad a Surtir'].to_dict()

In [ ]:
# Función para limpiar y convertir los datos de distancias
def limpiar_convertir_distancias(distances_df):
    # Función interna para limpiar los valores de distancia
    def clean_distance(x):
        # Si el valor es una cadena, elimina ' km' y convierte a float
        if isinstance(x, str):
            return float(x.replace(' km', ''))
        # Si el valor ya es numérico, simplemente lo retorna
        return x

    # Establece la primera columna como índice y aplica la función de limpieza a todos los elementos del DataFrame
    distances_cleaned_df = distances_df.set_index('Unnamed: 0').applymap(clean_distance)

    # Convierte el DataFrame limpio a un diccionario y lo retorna
    return distances_cleaned_df.to_dict()

In [ ]:
# Carga los datos desde el archivo Excel especificado en file_path
distances_df, demanda_df = cargar_datos(file_path)

# Limpia y convierte los datos de demanda a un diccionario
demanda_dict = limpiar_convertir_demanda(demanda_df)

# Limpia y convierte los datos de distancias a un diccionario
distances_dict = limpiar_convertir_distancias(distances_df)

In [ ]:
# Definir los centros de distribución
centros_distribucion = ['Coca Cola Lincoln', 'Coca Cola Universidad', 'Coca Cola Insurgentes', 'Coca Cola Guadalupe']

# Función para asignar sucursales a los centros de distribución basándose en la distancia más cercana
def asignar_sucursales_a_centros(demanda_dict, distances_dict, centros_distribucion):
    # Inicializa un diccionario para almacenar las asignaciones de sucursales a centros de distribución
    asignaciones = {centro: [] for centro in centros_distribucion}

    # Itera sobre cada sucursal en el diccionario de demanda
    for sucursal in demanda_dict.keys():
        # Encuentra el centro de distribución más cercano a la sucursal actual
        centro_cercano = min(centros_distribucion, key=lambda centro: distances_dict[centro][sucursal])
        # Asigna la sucursal al centro de distribución más cercano
        asignaciones[centro_cercano].append(sucursal)

    # Imprimir las asignaciones de sucursales a centros de distribución
    for centro, sucursales in asignaciones.items():
        print(f"Centro de distribución: {centro}")
        print("  Sucursales asignadas:")
        for sucursal in sucursales:
            print(f"    - {sucursal}")
        print()

    # Retorna el diccionario con las asignaciones
    return asignaciones

# Asignar sucursales a los centros de distribución utilizando la función definida
asignaciones = asignar_sucursales_a_centros(demanda_dict, distances_dict, centros_distribucion)

Centro de distribución: Coca Cola Lincoln
  Sucursales asignadas:
    - Pollo matón Lincoln
    - Pollo matón Aztlan
    - Pollo matón Solidaridad
    - Pollo matón Cumbres

Centro de distribución: Coca Cola Universidad
  Sucursales asignadas:
    - Pollo matón Girasoles
    - Pollo matón Escobedo
    - Pollo matón Concordia
    - Pollo matón Montes Berneses
    - Pollo matón San Nicolás
    - Pollo matón Rodrigo Gomez
    - Pollo matón Centrika
    - Pollo matón Centro
    - Pollo matón Santo Domingo
    - Pollo matón Cordillera de los Andes
    - Pollo matón Las Puentes

Centro de distribución: Coca Cola Insurgentes
  Sucursales asignadas:
    - Pollo matón Santa Catarina
    - Pollo matón Simon Bolivar
    - Pollo matón Lázaro Cárdenas

Centro de distribución: Coca Cola Guadalupe
  Sucursales asignadas:
    - Pollo matón Revolución
    - Pollo matón Lindavista



In [ ]:
# Función para encontrar la ruta del vecino más cercano
def nearest_neighbor_route(truck, distances):
    # Inicializa la ruta con el punto de partida del camión
    route = [truck[0]]
    # Crea un conjunto con las paradas restantes
    remaining_stops = set(truck[1:])

    # Imprime el punto de partida
    print("Punto de partida:", route[0])
    while remaining_stops:
        # Obtiene la última parada en la ruta actual
        last_stop = route[-1]
        # Encuentra la parada más cercana a la última parada
        next_stop = min(remaining_stops, key=lambda stop: distances[last_stop][stop])
        # Agrega la parada más cercana a la ruta
        route.append(next_stop)
        # Elimina la parada más cercana de las paradas restantes
        remaining_stops.remove(next_stop)

        # Imprime la ruta actual y las paradas restantes
        print("Ruta Actual:", route)
        print("Paradas faltantes:", remaining_stops)

    # Retorna la ruta completa
    return route

In [ ]:
# Función para asignar sucursales a los camiones basado en la capacidad
def assign_trucks_to_centers(asignaciones, demanda_dict, capacity):
    # Diccionario para almacenar los camiones asignados por centro de distribución
    trucks_per_center = {}

    # Iterar sobre cada centro de distribución y sus sucursales asignadas
    for centro, sucursales in asignaciones.items():
        trucks = []  # Lista para almacenar los camiones para el centro actual
        current_truck = []  # Lista para almacenar las sucursales asignadas al camión actual
        current_load = 0  # Variable para llevar la cuenta de la carga actual del camión

        # Iterar sobre cada sucursal asignada al centro actual
        for sucursal in sucursales:
            qty = demanda_dict[sucursal]  # Cantidad a surtir para la sucursal actual

            # Si la carga actual más la cantidad a surtir es menor o igual a la capacidad del camión
            if current_load + qty <= capacity:
                current_truck.append(sucursal)  # Agregar la sucursal al camión actual
                current_load += qty  # Actualizar la carga del camión
            else:
                # Si se excede la capacidad del camión, asignar el camión actual y empezar uno nuevo
                trucks.append(current_truck)
                print(f"Centro: {centro}, Camión asignado: {current_truck}, Carga total: {current_load}")
                current_truck = [sucursal]  # Nuevo camión empieza con la sucursal actual
                current_load = qty  # Actualizar la carga del nuevo camión

        # Agregar el último camión si tiene sucursales asignadas
        if current_truck:
            trucks.append(current_truck)
            print(f"Centro: {centro}, Camión asignado: {current_truck}, Carga total: {current_load}")

        # Guardar los camiones asignados para el centro actual
        trucks_per_center[centro] = trucks
        print(f"Centro: {centro}, Camiones asignados: {trucks}")

    # Retornar el diccionario con los camiones asignados por centro de distribución
    return trucks_per_center

# Asignar sucursales a los camiones dentro de cada centro de distribución
max_capacity = 3000
trucks_per_center = assign_trucks_to_centers(asignaciones, demanda_dict, max_capacity)

# Calcular las rutas para cada camión utilizando la heurística del vecino más cercano
routes_per_center = {centro: [nearest_neighbor_route(truck, distances_dict) for truck in trucks] for centro, trucks in trucks_per_center.items()}

Centro: Coca Cola Lincoln, Camión asignado: ['Pollo matón Lincoln', 'Pollo matón Aztlan', 'Pollo matón Solidaridad', 'Pollo matón Cumbres'], Carga total: 2430
Centro: Coca Cola Lincoln, Camiones asignados: [['Pollo matón Lincoln', 'Pollo matón Aztlan', 'Pollo matón Solidaridad', 'Pollo matón Cumbres']]
Centro: Coca Cola Universidad, Camión asignado: ['Pollo matón Girasoles', 'Pollo matón Escobedo', 'Pollo matón Concordia', 'Pollo matón Montes Berneses'], Carga total: 2740
Centro: Coca Cola Universidad, Camión asignado: ['Pollo matón San Nicolás', 'Pollo matón Rodrigo Gomez', 'Pollo matón Centrika', 'Pollo matón Centro', 'Pollo matón Santo Domingo'], Carga total: 2930
Centro: Coca Cola Universidad, Camión asignado: ['Pollo matón Cordillera de los Andes', 'Pollo matón Las Puentes'], Carga total: 995
Centro: Coca Cola Universidad, Camiones asignados: [['Pollo matón Girasoles', 'Pollo matón Escobedo', 'Pollo matón Concordia', 'Pollo matón Montes Berneses'], ['Pollo matón San Nicolás', 'Pol

In [ ]:
# Función para recalcular rutas con el retorno al centro de distribución
def recalcular_rutas_con_retorno(routes_per_center):
    # Diccionario para almacenar las rutas recalculadas con retorno al centro
    rutas_con_retorno = {}

    # Iterar sobre cada centro de distribución y sus rutas
    for centro, rutas in routes_per_center.items():
        rutas_con_retorno[centro] = []  # Inicializar lista para las nuevas rutas con retorno

        # Iterar sobre cada ruta asignada al centro actual
        for ruta in rutas:
            # Crear una nueva ruta que incluya el retorno al centro de distribución
            nueva_ruta = [centro] + ruta + [centro]
            # Agregar la nueva ruta a la lista de rutas con retorno
            rutas_con_retorno[centro].append(nueva_ruta)

            # Imprimir la ruta original y la nueva ruta con retorno
            print(f"Centro: {centro}, Ruta original: {ruta}, Ruta con retorno: {nueva_ruta}")

    # Retornar el diccionario con las rutas recalculadas
    return rutas_con_retorno

# Recalcular las rutas con el retorno al centro de distribución
rutas_con_retorno = recalcular_rutas_con_retorno(routes_per_center)

Centro: Coca Cola Lincoln, Ruta original: ['Pollo matón Lincoln', 'Pollo matón Solidaridad', 'Pollo matón Aztlan', 'Pollo matón Cumbres'], Ruta con retorno: ['Coca Cola Lincoln', 'Pollo matón Lincoln', 'Pollo matón Solidaridad', 'Pollo matón Aztlan', 'Pollo matón Cumbres', 'Coca Cola Lincoln']
Centro: Coca Cola Universidad, Ruta original: ['Pollo matón Girasoles', 'Pollo matón Escobedo', 'Pollo matón Concordia', 'Pollo matón Montes Berneses'], Ruta con retorno: ['Coca Cola Universidad', 'Pollo matón Girasoles', 'Pollo matón Escobedo', 'Pollo matón Concordia', 'Pollo matón Montes Berneses', 'Coca Cola Universidad']
Centro: Coca Cola Universidad, Ruta original: ['Pollo matón San Nicolás', 'Pollo matón Santo Domingo', 'Pollo matón Rodrigo Gomez', 'Pollo matón Centrika', 'Pollo matón Centro'], Ruta con retorno: ['Coca Cola Universidad', 'Pollo matón San Nicolás', 'Pollo matón Santo Domingo', 'Pollo matón Rodrigo Gomez', 'Pollo matón Centrika', 'Pollo matón Centro', 'Coca Cola Universidad']

In [ ]:
# Función para calcular la demanda total de cada ruta
def calcular_demanda_ruta(ruta, demanda_dict):
    # Utilizar la función sum() para sumar la demanda de cada sucursal en la ruta
    return sum(demanda_dict.get(sucursal, 0) for sucursal in ruta)

In [ ]:
# Función para calcular la distancia total de una ruta
def calculate_route_distance(route, distances):
    # Inicializa una variable para mantener la suma total de las distancias
    total_distance = 0

    # Itera sobre la lista de la ruta para calcular la distancia entre cada par de puntos consecutivos
    for i in range(len(route) - 1):
        # Suma la distancia entre los puntos consecutivos a la distancia total
        total_distance += distances[route[i]][route[i + 1]]

    # Retorna la distancia total calculada
    return total_distance

In [ ]:
# Función para calcular la distancia total de todas las rutas
def calculate_total_distance(routes, distances):
    # Inicializa el contador de la distancia total a cero
    total_distance = 0

    # Itera sobre cada ruta en la lista de rutas
    for route in routes:
        # Suma la distancia de cada ruta individual al total
        total_distance += calculate_route_distance(route, distances)

    # Devuelve la distancia total acumulada de todas las rutas
    return total_distance

In [ ]:
# Función para realizar un intercambio de dos nodos en una ruta, sin modificar el nodo inicial y final
def swap_two_nodes(route, i, k):
    # Verifica si los índices i o k corresponden al nodo inicial o final
    if i == 0 or k == 0 or i == len(route) - 1 or k == len(route) - 1:
        # Mensaje de error si se intenta intercambiar el nodo inicial o final
        print(f"No se puede intercambiar los nodos en las posiciones {i} y {k} porque uno de ellos es el nodo inicial o final.")
        return route  # Devuelve la ruta original sin cambios
    # Imprime la ruta antes del intercambio
    print(f"Ruta original: {route}")
    print(f"Intercambiando nodos en las posiciones {i} y {k}")
    # Realiza una copia de la lista para evitar modificar la original
    new_route = route[:]
    # Intercambia los nodos en las posiciones i y k
    new_route[i], new_route[k] = new_route[k], new_route[i]
    # Imprime la ruta después del intercambio
    print(f"Ruta después del intercambio: {new_route}")
    # Devuelve la nueva ruta con los nodos intercambiados
    return new_route

In [ ]:
# Función para aplicar búsqueda tabú con best improvement
def tabu_search_best_improvement(routes, distances, tabu_tenure=3, max_iterations=100):
    # Copiar las rutas actuales como las mejores encontradas hasta ahora
    best_routes = routes[:]
    # Calcular la distancia total de las rutas iniciales
    best_distance = calculate_total_distance(routes, distances)
    # Establecer las rutas actuales para comenzar la optimización
    current_routes = routes[:]
    # Inicializar la lista tabú que almacenará los movimientos prohibidos temporalmente
    tabu_list = []

    # Iterar hasta un máximo de iteraciones definido
    for iteration in range(max_iterations):
        # Inicializar el mejor movimiento local y su distancia como infinito
        best_local_move = None
        best_local_distance = float('inf')

        # Reportar en qué iteración del proceso estamos
        print(f"Iteración {iteration + 1}/{max_iterations}")
        # Iterar sobre cada ruta en la lista de rutas actuales
        for r in range(len(current_routes)):
            route = current_routes[r]
            # Considerar cada posible intercambio de nodos en la ruta, excepto el primero y último
            for i in range(1, len(route) - 1):
                for k in range(i + 1, len(route) - 1):
                    # Saltar este intercambio si está actualmente en la lista tabú
                    if (r, i, k) in tabu_list:
                        continue
                    # Realizar el intercambio y calcular la nueva distancia de la ruta
                    new_route = swap_two_nodes(route, i, k)
                    new_distance = calculate_route_distance(new_route, distances)
                    # Si la nueva ruta es mejor que la mejor encontrada en esta iteración, actualizar
                    if new_distance < best_local_distance:
                        best_local_distance = new_distance
                        best_local_move = (r, i, k, new_route)

        # Si se encontró un movimiento viable, realizar el cambio y actualizar la lista tabú
        if best_local_move:
            r, i, k, new_route = best_local_move
            current_routes[r] = new_route
            # Añadir el movimiento a la lista tabú
            tabu_list.append((r, i, k))
            # Mantener la lista tabú dentro del límite de tenure
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            # Calcular la nueva distancia total de las rutas
            current_distance = calculate_total_distance(current_routes, distances)
            # Reportar el mejor movimiento local y su efecto
            print(f"Mejor movimiento local: Ruta {r}, intercambiando {i} y {k}, nueva distancia: {best_local_distance}")
            print(f"Lista Tabú: {tabu_list}")

            # Si la distancia total actual es mejor que la mejor conocida, actualizar la mejor conocida
            if current_distance < best_distance:
                best_routes = current_routes[:]
                best_distance = current_distance
                print(f"Nueva mejor distancia encontrada: {best_distance}")

    # Al final del proceso, reportar la mejor distancia y rutas encontradas
    print(f"Mejor distancia final: {best_distance}")
    print(f"Mejores rutas finales: {best_routes}")
    return best_routes, best_distance

# Inicialización de estructuras para almacenar las rutas optimizadas y la distancia total mejorada
rutas_mejoradas = {}
distancia_mejorada_total = 0

# Iteración sobre cada centro de distribución
for centro in centros_distribucion:
    # Aplicar la búsqueda tabú con best improvement a las rutas del centro actual
    rutas_mejoradas[centro], distancia_mejorada = tabu_search_best_improvement(rutas_con_retorno[centro], distances_dict)

    # Acumular la distancia mejorada total
    distancia_mejorada_total += distancia_mejorada

Streaming output truncated to the last 5000 lines.
Lista Tabú: [(0, 2, 4), (0, 1, 2), (0, 3, 4)]
Iteración 81/100
Ruta original: ['Coca Cola Lincoln', 'Pollo matón Lincoln', 'Pollo matón Solidaridad', 'Pollo matón Aztlan', 'Pollo matón Cumbres', 'Coca Cola Lincoln']
Intercambiando nodos en las posiciones 1 y 3
Ruta después del intercambio: ['Coca Cola Lincoln', 'Pollo matón Aztlan', 'Pollo matón Solidaridad', 'Pollo matón Lincoln', 'Pollo matón Cumbres', 'Coca Cola Lincoln']
Ruta original: ['Coca Cola Lincoln', 'Pollo matón Lincoln', 'Pollo matón Solidaridad', 'Pollo matón Aztlan', 'Pollo matón Cumbres', 'Coca Cola Lincoln']
Intercambiando nodos en las posiciones 1 y 4
Ruta después del intercambio: ['Coca Cola Lincoln', 'Pollo matón Cumbres', 'Pollo matón Solidaridad', 'Pollo matón Aztlan', 'Pollo matón Lincoln', 'Coca Cola Lincoln']
Ruta original: ['Coca Cola Lincoln', 'Pollo matón Lincoln', 'Pollo matón Solidaridad', 'Pollo matón Aztlan', 'Pollo matón Cumbres', 'Coca Cola Lincoln']
I

In [ ]:
# Funcion para imprimir resultados
for centro in centros_distribucion:
    # Calcular las demandas y distancias de las rutas optimizadas para el centro actual
    demandas_rutas_mejoradas = [calcular_demanda_ruta(ruta, demanda_dict) for ruta in rutas_mejoradas[centro]]
    distancias_rutas_mejoradas = [calculate_route_distance(ruta, distances_dict) for ruta in rutas_mejoradas[centro]]

    # Imprimir las rutas mejoradas con sus demandas y distancias
    print(f"Centro de distribución: {centro}")
    for i, (ruta, demanda, distancia) in enumerate(zip(rutas_mejoradas[centro], demandas_rutas_mejoradas, distancias_rutas_mejoradas), 1):
        print(f"  Camión {i}:")
        print("    Ruta:", " → ".join(ruta))
        print("    Demanda total:", demanda, "kg")
        print("    Distancia de la ruta:", distancia, "km")
    print()

# Imprimir la distancia total recorrida después de la optimización
print("Distancia total recorrida:", distancia_mejorada_total, "km")

Centro de distribución: Coca Cola Lincoln
  Camión 1:
    Ruta: Coca Cola Lincoln → Pollo matón Lincoln → Pollo matón Solidaridad → Pollo matón Aztlan → Pollo matón Cumbres → Coca Cola Lincoln
    Demanda total: 2430 kg
    Distancia de la ruta: 30.299999999999997 km

Centro de distribución: Coca Cola Universidad
  Camión 1:
    Ruta: Coca Cola Universidad → Pollo matón Girasoles → Pollo matón Escobedo → Pollo matón Concordia → Pollo matón Montes Berneses → Coca Cola Universidad
    Demanda total: 2740 kg
    Distancia de la ruta: 35.9 km
  Camión 2:
    Ruta: Coca Cola Universidad → Pollo matón Rodrigo Gomez → Pollo matón San Nicolás → Pollo matón Santo Domingo → Pollo matón Centro → Pollo matón Centrika → Coca Cola Universidad
    Demanda total: 2930 kg
    Distancia de la ruta: 28.5 km
  Camión 3:
    Ruta: Coca Cola Universidad → Pollo matón Las Puentes → Pollo matón Cordillera de los Andes → Coca Cola Universidad
    Demanda total: 995 kg
    Distancia de la ruta: 17.2 km

Centro 